**Fake Video Detection Using LSTM**

In [1]:
#import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, TimeDistributed, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import glob

**Data Prerparation**

In [2]:
#load the data
data_dir = 'deepfakevedios/data'  

def load_video_paths(data_dir, label):
    video_paths = glob.glob(os.path.join(data_dir, label, '*.mp4'))
    return video_paths

train_real_videos = load_video_paths(os.path.join(data_dir, 'train'), 'real')
train_fake_videos = load_video_paths(os.path.join(data_dir, 'train'), 'fake')

val_real_videos = load_video_paths(os.path.join(data_dir, 'val'), 'real')
val_fake_videos = load_video_paths(os.path.join(data_dir, 'val'), 'fake')

test_real_videos = load_video_paths(os.path.join(data_dir, 'test'), 'real')
test_fake_videos = load_video_paths(os.path.join(data_dir, 'test'), 'fake')


**Functions**

In [3]:
def extract_frames(video_path, max_frames=20, img_size=(128, 128)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, frame_count // max_frames)
    
    for i in range(0, frame_count, frame_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, img_size)
            frames.append(frame)
        if len(frames) >= max_frames:
            break
    cap.release()
    return np.array(frames)

In [4]:
def load_video_data(video_paths, max_frames=20, img_size=(128, 128)):
    videos = []
    for video_path in video_paths:
        frames = extract_frames(video_path, max_frames, img_size)
        if len(frames) == max_frames:
            videos.append(frames)
    return np.array(videos)

In [5]:
train_videos_real = load_video_data(train_real_videos)
train_videos_fake = load_video_data(train_fake_videos)
train_videos = np.concatenate((train_videos_real, train_videos_fake), axis=0)
train_labels = np.array([0] * len(train_videos_real) + [1] * len(train_videos_fake))

val_videos_real = load_video_data(val_real_videos)
val_videos_fake = load_video_data(val_fake_videos)
val_videos = np.concatenate((val_videos_real, val_videos_fake), axis=0)
val_labels = np.array([0] * len(val_videos_real) + [1] * len(val_videos_fake))

test_videos_real = load_video_data(test_real_videos)
test_videos_fake = load_video_data(test_fake_videos)
test_videos = np.concatenate((test_videos_real, test_videos_fake), axis=0)
test_labels = np.array([0] * len(test_videos_real) + [1] * len(test_videos_fake))

**CNN Model Building**

In [ ]:
IMG_SIZE = (128, 128)
MAX_FRAMES = 20
BATCH_SIZE = 8

model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(MAX_FRAMES, IMG_SIZE[0], IMG_SIZE[1], 3)),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),
    
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),
    
    TimeDistributed(Flatten()),
    
    LSTM(64, return_sequences=False),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [7]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

**Training**

In [ ]:
history = model.fit(
    train_videos, train_labels,
    batch_size=BATCH_SIZE,
    epochs=30,
    validation_data=(val_videos, val_labels)
)

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 20, 126, 126, 32)    │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 20, 126, 126, 32)    │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 20, 63, 63, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 20, 61, 61, 64)      │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 20, 61, 61, 64)      │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 20, 30, 30, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_6 (TimeDistributed) │ (None, 20, 57600)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │      14,762,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,790,979 (56.42 MB)

 Trainable params: 14,790,529 (56.42 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

**Classification Report**

In [ ]:
test_loss, test_acc = model.evaluate(test_videos, test_labels, batch_size=BATCH_SIZE)
print(f'Test Accuracy: {test_acc}')

y_pred = model.predict(test_videos)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

print(classification_report(test_labels, y_pred_classes))


**Performance measures**

**Accuracy and Loss Curves during Training, Validation**

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

**Confusion Matrix**

In [ ]:
conf_matrix = confusion_matrix(test_labels, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [14]:
model.save('fake_video_detection.h5')

In [15]:
loaded_model = tf.keras.models.load_model('fake_video_detection.h5')

In [16]:
def extract_frames_from_folder(folder_path, max_frames=20, img_size=(128, 128)):
    video_data = {}
    for video_file in os.listdir(folder_path):
        if video_file.endswith('.mp4'): 
            video_path = os.path.join(folder_path, video_file)
            frames = extract_frames(video_path, max_frames, img_size)
            if len(frames) == max_frames:
                video_data[video_file] = frames
    return video_data

In [17]:
def classify_videos_in_folder(model, folder_path, max_frames=20, img_size=(128, 128), threshold=0.5):
    video_data = extract_frames_from_folder(folder_path, max_frames, img_size)
    results = {}
    
    for video_file, frames in video_data.items():
        frames = np.expand_dims(frames, axis=0)
        prediction = model.predict(frames)
        predicted_class = 'Real' if prediction > threshold else 'Fake'
        results[video_file] = predicted_class
    
    return results

In [18]:
# Example usage
folder_path = 'deepfakevedios/data/test/real'  
results = classify_videos_in_folder(model, folder_path)

for video_file, classification in results.items():
    print(f'{video_file}: {classification}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
# Example usage
folder_path = 'deepfakevedios/data/test/fake'  
results = classify_videos_in_folder(model, folder_path)

for video_file, classification in results.items():
    print(f'{video_file}: {classification}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 